# Training Notebook

この Notebook は、もともとの `train.py` のコードを Notebook 用にセル分割したものです。以下の各セルを順次実行することで、学習プロセスを開始できます。

In [1]:
import os
import logging
import random
from pathlib import Path
import datetime
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
from dotenv import load_dotenv

load_dotenv()

from datasets import disable_progress_bars
disable_progress_bars()
from datasets import load_dataset

from huggingface_hub import snapshot_download

from schedulefree import RAdamScheduleFree
from sentence_transformers import SentenceTransformer

from gemma.config import GemmaConfig, get_model_config
from gemma.model import GemmaForCausalLM
from VAEs.LinearVAE import LinearVAE
from VAEs.VAE import VAE
from VAEs.Perceptron import Perceptron

torch.set_float32_matmul_precision('high')

/home/yut/sail-develop/vec_gemma/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
class TrainingConfig:
    def __init__(self):
        # データ設定
        self.batch_size = 4
        self.max_seq_len = 256
        self.num_workers = 4
        
        # 最適化設定
        self.lr = 1e-4
        self.num_epochs = 10
        self.grad_accum_steps = 4
        self.beta_init = 0.00
        self.beta_max = 0
        self.beta_step = 1e-6
        self.crop_lambda = 0.2

        # モデル設定
        self.bert_model_name = "cl-nagoya/ruri-large"
        self.gemma_model_size = "2b-v2"
        self.vae_hidden_dim = 512
        self.vae_latent_dim = 128
        
        # 生成設定
        self.sample_interval = 1000
        self.num_samples = 3
        self.max_gen_length = 100
        self.generation_temp = 0.1
        self.generation_top_p = 0.2
        self.generation_top_k = 2
        
        self.ckpt_interval = 5000
        # パス設定
        self.log_dir = f"./logs/{datetime.datetime.now()}/"
        self.checkpoint_dir = "./checkpoints"
        self.dataset_path = "AhmedSSabir/Japanese-wiki-dump-sentence-dataset"
        
        # 初期化
        self._setup_directories()
        
    def _setup_directories(self):
        Path(self.log_dir).mkdir(parents=True, exist_ok=True)
        Path(self.checkpoint_dir).mkdir(parents=True, exist_ok=True)


def setup_logging(log_dir):
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler(os.path.join(log_dir, "training.log")),
            logging.StreamHandler()
        ]
    )


def prepare_dataset(config):
    def collate_fn(batch):
        return [item['text'] for item in batch]
    
    dataset = load_dataset(config.dataset_path, cache_dir="./.datasets")

    train_loader = DataLoader(
        dataset['train'].with_format("torch"),
        batch_size=config.batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=config.num_workers,
        pin_memory=True
    )
    return train_loader


def initialize_models(config, device):
    # BERTモデル
    bert_model = SentenceTransformer(config.bert_model_name)
    bert_model.requires_grad_(False)
    bert_model = bert_model.to(device)
    
    # Gemmaモデル
    snapshot_dir = snapshot_download(repo_id='google/gemma-2-2b-jpn-it-pytorch')

    # Ensure that the tokenizer is present
    tokenizer_path = os.path.join(snapshot_dir, 'tokenizer.model')
    assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

    # Ensure that the checkpoint is present
    ckpt_path = os.path.join(snapshot_dir, f'model.ckpt')
    assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'
    

    gemma_model_config = get_model_config("2b-v2")
    gemma_model_config.tokenizer = tokenizer_path
    
    # Instantiate the model and load the weights.
    torch.set_default_dtype(gemma_model_config.get_dtype())
    gemma_model = GemmaForCausalLM(gemma_model_config)
    gemma_model.requires_grad_(False)
    gemma_model.load_weights(ckpt_path)
    gemma_model = gemma_model.to(device).eval()
    
    # VAEモデル
    vae_model = Perceptron(
        bert_model.get_sentence_embedding_dimension(),
        gemma_model.config.hidden_size,
        hidden_dim=config.vae_hidden_dim,
        latent_dim=config.vae_latent_dim
    ).to(device)
    
    return bert_model, gemma_model, vae_model


def generate_and_log_samples(vae_model, bert_model, gemma_model, device, writer, global_step, config):
    vae_model.eval()
    sample_texts = [
        "人工知能の未来について",
        "量子コンピュータの可能性",
        "ディープラーニングの応用分野",
        "自然言語処理の最新動向",
        "ロボット工学の進化",
        "一時期所長となる。",
        "京都府京都市に生まれる。",
        "卒業後は文章を書く仕事がしたいと1994年に報知新聞社にスポーツ記者として勤務し、高校野球やゴルフを取材する。",
        "その後、全てをリセットするためにタンザニア・ダルエスサラーム大学に留学し、スワヒリ語科で学ぶ。",
        "29歳の時に新人賞の最終候補に残るが、その後結婚し、出産したことで小説を書く余裕を無くしてしまう。",
        "本形式は、192形を改称して生まれた形式である。",
        "ただし、現実にはこの変更を受けたのは数両程度である。"
    ]
    
    with torch.no_grad():
        prep_texts = ["文章: " + text for text in sample_texts]
        
        # BERTで埋め込みを取得
        bert_embeddings = bert_model.encode(
            prep_texts, 
            convert_to_tensor=True,
            device=device,
            show_progress_bar=False
        ).to(torch.bfloat16)
        
        # VAEで埋め込みを変換
        vae_output, _, _ = vae_model(bert_embeddings)
        
        # Gemmaでテキスト生成
        generated_texts = []
        for i in range(len(sample_texts)):
            try:
                embedding = vae_output[i].unsqueeze(0).unsqueeze(0)
                generated = gemma_model.generate_with_initial_embedding(
                    initial_embedding=embedding,
                    device=device,
                    output_len=config.max_gen_length,
                    temperature=config.generation_temp,
                    top_p=config.generation_top_p,
                    top_k=config.generation_top_k,
                    instructions=("<start_of_turn>user\n \"", "\"\nこれをそのままの意味で出力してください。<end_of_turn>\n<start_of_turn>model\n")
                )
                generated_texts.append(generated[0])
            except Exception as e:
                generated_texts.append(f"生成エラー: {str(e)}")
        
        # ログに記録
        log_text = "\n\n".join([
            f"入力: {sample_texts[i]}\n生成結果: {generated_texts[i]}" 
            for i in range(len(sample_texts))
        ])
        
        writer.add_text("生成サンプル", log_text, global_step)
        logging.info(f"\n=== ステップ {global_step} の生成サンプル ===\n{log_text}\n")
    
    vae_model.train()


def train(config, vae_model, bert_model, gemma_model, optimizer, device, start_epoch=0, initial_beta=0.1):
    writer = SummaryWriter(config.log_dir)
    setup_logging(config.log_dir)
    
    instructions = [
        ("<start_of_turn>user以下の内容をそのまま再現してください:\"", "\"\n<end_of_turn><start_of_turn>model\n"),
    ]

    best_loss = float('inf')
    beta = initial_beta
    optimizer.train()

    for epoch in range(start_epoch, 100):
        logging.info(f"エポック {epoch+1}/{config.num_epochs}")
        train_loader = prepare_small_dataset(config)
        total_steps = len(train_loader)
        progress_bar = tqdm(
            train_loader,
            total=total_steps,
            desc=f"エポック {epoch+1}/{config.num_epochs}",
            bar_format="{l_bar}{bar:20}{r_bar}{bar:-10b}",
            dynamic_ncols=True
        )
        for step, batch_texts in enumerate(progress_bar):
            with torch.no_grad():
                rep_texts = ["文章: " + text for text in batch_texts]
                bert_embeddings = bert_model.encode(
                    rep_texts, 
                    convert_to_tensor=True,
                    device=device,
                    show_progress_bar=False
                ).to(torch.bfloat16)
            
            with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                vae_output, mu, logvar = vae_model(bert_embeddings)
                
                kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
                kl_div_per_batch = kl_div / bert_embeddings.size(0)
                kl_div = torch.maximum(
                    kl_div_per_batch,
                    torch.tensor(config.crop_lambda, device=kl_div.device, dtype=kl_div.dtype)
                )
                
                recon_loss = gemma_model.forward_teacher_forcing(
                    vae_output, 
                    batch_texts,
                    max_seq_len=config.max_seq_len,
                    instructions=instructions[random.randint(0, len(instructions)-1)],
                )
                
                loss = recon_loss + beta * kl_div

            loss.backward()
            
            if (step + 1) % config.grad_accum_steps == 0:
                optimizer.step()
            
            writer.add_scalar("損失/訓練", loss.item(), epoch*len(train_loader)+step)
            writer.add_scalar("損失/再構成", recon_loss.item(), epoch*len(train_loader)+step)
            writer.add_scalar("損失/KL", kl_div.item(), epoch*len(train_loader)+step)
            writer.add_scalar("Beta", beta, epoch*len(train_loader)+step)
            
            progress_bar.set_postfix({
                "loss": f"{loss.item():.4f}",
                "recon": f"{recon_loss.item():.4f}",
                "kl": f"{kl_div.item():.4f}",
                "beta": f"{beta:.4f}"
            })
            optimizer.eval()

            #if step % config.sample_interval == 0:
            #    generate_and_log_samples(
            ##        vae_model=vae_model,
            #        bert_model=bert_model,
            #        gemma_model=gemma_model,
            #        device=device,
            #        writer=writer,
            #        global_step=epoch*len(train_loader)+step,
            #        config=config,
            #    )
            if step % config.ckpt_interval == 0:
                checkpoint = {
                    "epoch": epoch-1,
                    "model_state": vae_model.state_dict(),
                    "optimizer_state": optimizer.state_dict(),
                    "loss": loss.item(),
                    "beta": beta,
                    "settings": config.__dict__
                }
                torch.save(checkpoint, os.path.join(config.checkpoint_dir, f"checkpoint_epoch_{epoch+1}_step_{datetime.datetime.now()}_{step}.pt"))
                torch.save(vae_model.state_dict(), os.path.join(config.checkpoint_dir, "latest_model.pt"))
            optimizer.train()
            beta = min(config.beta_max, beta + config.beta_step)
        
        #generate_and_log_samples(
        #    vae_model=vae_model,
        #    bert_model=bert_model,
        #    gemma_model=gemma_model,
        #    device=device,
        #    writer=writer,
        #    global_step=(epoch+1)*len(train_loader),
        #    config=config
        #)
        
        #checkpoint = {
        #    "epoch": epoch,
        #    "model_state": vae_model.state_dict(),
        #    "optimizer_state": optimizer.state_dict(),
        #    "loss": loss.item(),
        #    "beta": beta,
        #    "settings": config.__dict__
        #}
        
        #torch.save(checkpoint, os.path.join(config.checkpoint_dir, f"checkpoint_epoch_{epoch+1}_{datetime.datetime.now()}.pt"))
        
        if loss.item() < best_loss:
            best_loss = loss.item()
            torch.save(vae_model.state_dict(), os.path.join(config.checkpoint_dir, f"best_model{datetime.datetime.now()}.pt"))

In [4]:
# --- 初期化＆チェックポイントの読み込み（必要に応じて） ---

config = TrainingConfig()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bert_model, gemma_model, vae_model = initialize_models(config, device)
vae_model.type(torch.bfloat16)
gemma_model.type(torch.bfloat16)
optimizer = RAdamScheduleFree(vae_model.parameters(), lr=config.lr)

# チェックポイントから再開したい場合は、以下 resume にパスを設定してください。なければ None のままでOK
resume = None  # 例: "./checkpoints/your_checkpoint.pt"
if resume:
    checkpoint = torch.load(resume, map_location=device)
    vae_model.load_state_dict(checkpoint["model_state"])
    optimizer.load_state_dict(checkpoint["optimizer_state"])
    start_epoch = checkpoint["epoch"] + 1
    beta = checkpoint["beta"]
    if "settings" in checkpoint:
        config.__dict__.update(checkpoint["settings"])
    logging.info(f"Resuming training from epoch {start_epoch}")
else:
    start_epoch = 0
    beta = config.beta_init

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 70492.50it/s]
/home/yut/sail-develop/vec_gemma/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1132: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at ../aten/src/ATen/native/Copy.cpp:308.)
  return self._apply(lambda t: t.type(dst_type))


In [2]:
def prepare_small_dataset(config):
    def collate_fn(batch):
        return [item['text'] for item in batch]
    
    dataset = load_dataset(config.dataset_path, cache_dir="./.datasets")

    train_loader = DataLoader(
        dataset['train'].shuffle(seed=42).select(range(100)),
        batch_size=config.batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=config.num_workers,
        pin_memory=True
    )
    return train_loader

In [4]:
text = "あああああああ"
bert_emb = bert_model.encode([text], convert_to_tensor=True, device=device).to(torch.bfloat16)
vae_output, _, _ = vae_model(bert_emb)
gemma_model.train()

loss = gemma_model.forward_teacher_forcing(
    vae_output, [text], max_seq_len=config.max_seq_len,
    instructions=("<start_of_turn>user\n \"", "\"\nこれをそのままの意味で出力してください。<end_of_turn>\n<start_of_turn>model\n")
)
print(loss.item())
from torchviz import make_dot

# 例: loss の計算が終わった後
dot = make_dot(loss, params=dict(vae_model.named_parameters()))
dot.render("loss_graph", format="svg")  # loss_graph.png として保存される

with torch.autograd.set_detect_anomaly(True):
    loss.backward()
    for name, param in vae_model.named_parameters():
        if param.grad is not None:
            print(name, param.grad.norm().item())
        else:
            print(name, "None")


6.1552734375
fc1.weight 25.25
fc1.bias 0.9765625
fc2.weight 8.6875
fc2.bias 2.453125


In [16]:
# --- トレーニング開始 ---
torch.cuda.empty_cache()
train(
    config=config,
    vae_model=vae_model,
    bert_model=bert_model,
    gemma_model=gemma_model,
    optimizer=optimizer,
    device=device,
    start_epoch=start_epoch,
    initial_beta=beta
)

2025-02-11 00:44:31,386 - INFO - エポック 1/10
エポック 1/10:   0%|                    | 0/25 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 0 has a total capacity of 15.58 GiB of which 24.25 MiB is free. Including non-PyTorch memory, this process has 14.63 GiB memory in use. Of the allocated memory 14.20 GiB is allocated by PyTorch, and 276.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

学習が開始されました。ログや TensorBoard を確認してください。

In [14]:
small_loader = prepare_small_dataset(config)
text=small_loader.dataset[0]['text']

bert_emb = bert_model.encode([text], convert_to_tensor=True, device=device).to(torch.bfloat16)
vae_output, _, _ = vae_model(bert_emb)
gemma_model.generate_with_initial_embedding(
    initial_embedding=vae_output,
    device=device,
    output_len=config.max_gen_length,
    temperature=config.generation_temp,
    top_p=config.generation_top_p,
    top_k=2,
    instructions=("<start_of_turn>user以下の内容をそのまま再現してください:\"", "\"\n<end_of_turn><start_of_turn>model\n")
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 39.45it/s]


['"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n"\n\n']